In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Getting lemmas and metadata together****

In [2]:
%%time
# open file in read mode
with open('/kaggle/input/utf8tokenizedspeeches/TokenizedSpeeches_utf-8.csv', 'r') as read_obj:
#with open('TokenizedSpeeches_utf-8.csv', 'r') as read_obj:

    lemmasList = []
    
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader: 
        lemmasList.append(row)
    print(len(lemmasList), 'Rows read')

In [3]:
%%time
#create dataframe from the lemmas extracted from csv
dictOfLemmas = {'Lemmas': lemmasList}
lemmasDf = pd.DataFrame(dictOfLemmas)
lemmasDf

In [4]:
with open('/kaggle/input/hansard-speeches-lemmatized/hansard-speeches-post2010.pkl', 'rb') as f:
    df = pickle.load(f)

In [5]:
#since index was missing values and didn't match with the lemmasDf index
df = df.reset_index(drop=True)
df = df.join(lemmasDf)
df['Lemmas']

# Dividing and training corpus before and after the Brexit referendum

In [6]:
# Split data based on the Brexit referendum event before and after period
eventDate = '2016-06-23 23:59:59'
df_t1 = df[df['date']<= eventDate]
df_t2 = df[df['date']> eventDate]

# Choosing intersecting vocabulary and Aligning models 

In [7]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        if(len(m.wv.key_to_index)==135):
            print('Common vocab is', common_vocab)
        
    return (m1,m2)

In [8]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
        
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

# Splitting speeches by MPs and time

In [9]:
%%time
# Lets only consider the MPs who have speeches in both T1 & T2 time periods, intersection between two DFs 

t1List = pd.unique(df_t1['mnis_id']).tolist()
t2List = pd.unique(df_t2['mnis_id']).tolist()

intersectedList = list(set(t1List).intersection(t2List))

len(intersectedList)
# 651 MPs to be considered

In [10]:
# Now modify original df_t1 and df_t2 DFs to only contain speeches from the common MPs 

df_t1 = df_t1[df_t1['mnis_id'].isin(intersectedList)]
df_t2 = df_t2[df_t2['mnis_id'].isin(intersectedList)]
df_t2['Lemmas']

In [11]:
%%time
# Original code to divide by MP and time
# This creates DFs with vocabs of each MP split per speech

'''dictSpeechesByMp = {}

for mpId in intersectedList:
    for dfTime in ['df_t1','df_t2']:
        dfName = dfTime + '_'+ mpId
        if(dfTime == 'df_t1'):
            dictSpeechesByMp[dfName]=df_t1[df_t1['mnis_id']==mpId]
        elif (dfTime == 'df_t2'):
            dictSpeechesByMp[dfName]=df_t2[df_t2['mnis_id']==mpId]
            '''

In [12]:
%%time
# New code for dividing corpus by time and MPs 

 #   -x  -x  -x  Integrates all lemmas to map per MP - one vocab per MP per time period (eventually two vocabs per MP across T1 & T2)  -x  -x  -x  -x  -x

dictSpeechesByMp = {}

for mpId in intersectedList:
  
    for dfTime in ['df_t1','df_t2']:
        tempDf = pd.DataFrame()
        tempList = []
        Lemmas =[]
        dfName = dfTime + '_'+ mpId

        if(dfTime == 'df_t1'):
            tempDf = df_t1[df_t1['mnis_id']==mpId]
        elif (dfTime == 'df_t2'):
            tempDf = df_t2[df_t2['mnis_id']==mpId] 
            
        tempList.extend(tempDf['Lemmas'].to_list())
        party = tempDf['party'].iat[0]
    
        #Flatten the list so it's not a list of lists
        tempList = [item for sublist in tempList for item in sublist]
        
        tempDf = pd.DataFrame([[mpId, party, tempList]],columns=['mnis_id', 'party', 'Lemmas'])
        dictSpeechesByMp[dfName]= tempDf
            

In [13]:
len(dictSpeechesByMp.keys())

In [14]:
# Do not run, for verification only

len(dictSpeechesByMp.keys()) 
# Result 1302 

df_t1['mnis_id'].value_counts()
df_t2['mnis_id'].value_counts()
# Result is 651 each - hence all covered in our dictionary DFs 

In [35]:
# Conducting analysis of vocabularies of MPs (1302 vocabs and entities)

vocabDf = pd.DataFrame(columns = ['mnis_id', 'party', 'Lemmas'])

for val in dictSpeechesByMp.values():   
    vocabDf = vocabDf.append(val)

vocabDf['LengthLemmas'] = vocabDf.Lemmas.map(len)
vocabDf.agg(Max=('LengthLemmas', max), Min=('LengthLemmas', 'min'), Mean=('LengthLemmas', np.mean))

In [36]:
# Plotting vocab per MP per timeperiod distribution

plt.hist(vocabDf['LengthLemmas'], bins = 50)
plt.show()

In [37]:
# Checking for cutoff for vocab size
# Choosing all vocabs which have more than 10000 words

vocabDf = vocabDf[vocabDf['LengthLemmas']>10000] 
#- 921 rows

#vocabDf[vocabDf['LengthLemmas']>1000] = 1287 rows
#vocabDf[vocabDf['LengthLemmas']>5000] - 1123 rows


In [38]:
# We should only keep speeches by those MPs which have occurences in both T1 & T2 since comparison in their vocab is to be made here

#list(vocabDf['mnis_id'].value_counts().value_counts())
#340 MPs have speeches in both T1 and T, 241 have speech(es) in one of the time periods

mpCountDict = vocabDf['mnis_id'].value_counts().to_dict()

vocabDf['mpCountInTimes'] = vocabDf['mnis_id'].map(mpCountDict)

#Drop the MPs with selected vocab in only one time interval

vocabDf=vocabDf[vocabDf['mpCountInTimes']==2]

# For manual verification
'''vocabDf[vocabDf['mnis_id']=='3942']
3992, 3942 - only one instance of each
4263 - 2 instances'''

In [41]:
vocabDf.shape

In [42]:
# Now backtrack and update dictionary to only keep DFs of these 921 rows

# Creating dict key names to keep from our filtered DF mnis IDs

mnis_id_list = list(vocabDf['mnis_id'].value_counts().to_dict().keys())

list1 = ['df_t1_'+item for item in mnis_id_list]
list2 = ['df_t2_'+item for item in mnis_id_list]
mnis_id_list = list1 + list2

In [43]:
# Dropping key-value pairs from dictionary where the key doesn't match

for k,v in list(dictSpeechesByMp.items()):
    if (k not in mnis_id_list):
        del dictSpeechesByMp[k]

In [64]:
%%time

#  - x - x - x - CREATING & SAVING WORD2VEC MODELS FOR THE 680 MODELS - x - x - x 

dictOfModels = {}
shutil.rmtree('./models-by-mp-and-time')
os.makedirs('./models-by-mp-and-time')
models_folder = './models-by-mp-and-time'
count = 1

for dframe in dictSpeechesByMp: 
    
    # Doing in batches since notebook RAM crashes

        
    #if (count<=int(len(dictSpeechesByMp.keys())/2)):
    if (count > int(len(dictSpeechesByMp.keys())/2)):
        
        model = gensim.models.Word2Vec(dictSpeechesByMp[dframe]['Lemmas'], min_count=1, vector_size=300, window = 5, sg = 1)


        # Also saving model in a dict and exporting

        modelName ='model_'+ dframe
        print('model number', count, modelName)

        dictOfModels[dframe] = model
        model.save(os.path.join(models_folder, modelName))
        count = count +1
    
    else:
        #continue in case of processing second batch, breaking in case of first batch
        #break#
        count = count+1
        continue


In [77]:
'''%%time
# Commenting since we're going to load and start with alignment of models with a fresh notebook
functools.reduce(smart_procrustes_align_gensim, list(dictOfModels.values()))'''

In [65]:
%%time
!zip -r file.zip /kaggle/working/models-by-mp-and-time

In [49]:
# To delete if need be
'''import shutil
shutil.rmtree('./models-by-mp-and-time')

os.remove('./file.zip')'''